In [1]:
from pymc3 import *
from sklearn.metrics import mean_absolute_error

Using gpu device 0: GeForce 940MX (CNMeM is disabled, cuDNN 5105)
/home/him/anaconda2/envs/theano-gpu/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [43]:
import pandas as pd
true = pd.read_csv('../../train.csv')
true = true['loss'].as_matrix()
type(true)

numpy.ndarray

In [44]:
tt = pd.read_csv('keras-train-1111.7.csv')
tt = tt.sort('id')
tt1 = pd.read_csv('keras-train-1114.csv')
tt1 = tt1.sort('id')
tt2 = pd.read_csv('keras-train-unk1.csv')
tt2 = tt2.sort('id')
tt3 = pd.read_csv('keras-train-unk2.csv')
tt3 = tt3.sort('id')
tt4 = pd.read_csv('keras-train-unk3.csv')
tt4 = tt4.sort('id')

/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [45]:
m1 = pd.read_csv('xgb-train-1124.75.csv')['loss'].as_matrix()
m2 = pd.read_csv('xgb-train-1127.27.csv')['loss'].as_matrix()
k1 = tt['loss'].as_matrix()
k2 = tt1['loss'].as_matrix()
k3 = tt2['loss'].as_matrix()
k4 = tt3['loss'].as_matrix()
k5 = tt4['loss'].as_matrix()

In [46]:
from sklearn.metrics import mean_absolute_error
print mean_absolute_error(true,m1)
print mean_absolute_error(true,m2)
print mean_absolute_error(true,k1)
print mean_absolute_error(true,k2)
print mean_absolute_error(true,k3)
print mean_absolute_error(true,k4)
print mean_absolute_error(true,k5)

1124.75950017
1127.27187837
1130.58667325
1132.58385255
1135.48586825
1137.42316001
1134.46582936


In [47]:
print mean_absolute_error(true,m1*0.16+m2*0.14+k1*0.14+k2*0.14+k3*0.14+k4*0.14+k5*0.14)

1121.63972461


In [48]:
import numpy as np
x_train = np.vstack((m1,m2,k1,k2,k5)).T
y = true

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
%matplotlib inline
clfLR = LinearRegression(fit_intercept=False)
clfLR.fit(x_train,y)
print clfLR.coef_
print mean_absolute_error(true,clfLR.predict(x_train))

In [12]:
from sklearn.ensemble import GradientBoostingRegressor
clfGB = GradientBoostingRegressor(max_depth=6,loss='lad')
clfGB.fit(x_train,y)
print mean_absolute_error(true,clfGB.predict(x_train))

1104.02856099


In [18]:
pred = clfGB.predict(x_test)
sub = pd.read_csv('xgb-test-1124.75.csv')
sub['loss'] = pred
sub.to_csv('GBM-Stacking-6-Models.csv',index=False)

In [49]:
tt = pd.read_csv('keras-test-1111.7.csv')
tt = tt.sort('id')
tt1 = pd.read_csv('keras-test-1114.csv')
tt1 = tt1.sort('id')
tt2 = pd.read_csv('keras-test-unk1.csv')
tt2 = tt2.sort('id')
tt3 = pd.read_csv('keras-test-unk2.csv')
tt3 = tt3.sort('id')
tt4 = pd.read_csv('keras-test-unk3.csv')
tt4 = tt4.sort('id')
m1 = pd.read_csv('xgb-test-1124.75.csv')['loss'].as_matrix()
m2 = pd.read_csv('xgb-test-1127.27.csv')['loss'].as_matrix()
k1 = tt['loss'].as_matrix()
k2 = tt1['loss'].as_matrix()
k3 = tt2['loss'].as_matrix()
k4 = tt3['loss'].as_matrix()
k5 = tt4['loss'].as_matrix()

x_test = np.vstack((m1,m2,k1,k2,k5)).T

/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [50]:
import xgboost as xgb

In [51]:
dtrain = xgb.DMatrix(x_train, label=np.log(true))
dtest = xgb.DMatrix(x_test)

In [52]:
xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.8,
    'silent': 1,
    'subsample': 0.6,
    'learning_rate': 0.01,
    'objective': 'reg:linear',
    'max_depth': 4,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'eval_metric': 'mae',
}

In [53]:
def xg_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y), np.exp(yhat))

In [54]:
SEED = 0
res = xgb.cv(xgb_params, dtrain, num_boost_round=1000, nfold=4, seed=SEED, stratified=False,
             early_stopping_rounds=25, verbose_eval=10, show_stdv=True, feval=xg_eval_mae, maximize=False)

[0]	train-mae:3035.56+4.23313	test-mae:3035.56+12.6993
[10]	train-mae:3033.82+4.23261	test-mae:3033.82+12.6989
[20]	train-mae:3030.81+4.23152	test-mae:3030.81+12.6984
[30]	train-mae:3025.89+4.22942	test-mae:3025.89+12.697
[40]	train-mae:3018.26+4.22558	test-mae:3018.26+12.695
[50]	train-mae:3007+4.21896	test-mae:3007+12.6908
[60]	train-mae:2991.12+4.20883	test-mae:2991.12+12.6851
[70]	train-mae:2969.62+4.19316	test-mae:2969.62+12.6761
[80]	train-mae:2941.58+4.17138	test-mae:2941.58+12.6611
[90]	train-mae:2906.24+4.14387	test-mae:2906.24+12.6379
[100]	train-mae:2863.06+4.09933	test-mae:2863.05+12.6107
[110]	train-mae:2811.76+4.04523	test-mae:2811.76+12.5781
[120]	train-mae:2752.4+3.9732	test-mae:2752.4+12.5449
[130]	train-mae:2685.34+3.89902	test-mae:2685.35+12.501
[140]	train-mae:2611.24+3.81691	test-mae:2611.26+12.4632
[150]	train-mae:2530.94+3.73078	test-mae:2530.98+12.4448
[160]	train-mae:2445.51+3.62711	test-mae:2445.58+12.4244
[170]	train-mae:2356.24+3.53621	test-mae:2356.33+12.33

In [55]:
best_nrounds = res.shape[0] - 1
print best_nrounds

999


In [56]:
cv_mean = res.iloc[-1, 0]
cv_std = res.iloc[-1, 1]
print('Ensemble-CV: {0}+{1}'.format(cv_mean, cv_std))

Ensemble-CV: 1123.05014025+5.75822213213


In [57]:
gbdt = xgb.train(xgb_params, dtrain, best_nrounds)

In [58]:
submission = pd.read_csv('../submission.csv')
submission.iloc[:, 1] = np.exp(gbdt.predict(dtest))
submission.to_csv('xgstacker_unsubmitted_v2.sub.csv', index=None)

In [59]:
print mean_absolute_error(np.exp(gbdt.predict(dtrain)),true)

1116.33607044


In [21]:
print mean_absolute_error(true,0.5*m1+0.5*k1)

1120.31821199


In [24]:
for i in range(1,11):
    print i,mean_absolute_error(true,i*0.1*m1+(10-i)*0.1*k1)

1 1127.35448493
2 1124.72814865
3 1122.69282074
4 1121.2183035
5 1120.31821199
6 1120.02294573
7 1120.34295071
8 1121.2345667
9 1122.70307992
10 1124.75950017


In [31]:
temp,best = None,None
for i in np.arange(0,1.1,0.1):
    for j in np.arange(0,1.1-i,0.1):
        for k in np.arange(0,1.1-i-j,0.1):
            for l in np.arange(0,1.1-i-j-k,0.1):
                if best is None:
                    best = mean_absolute_error(true,i*m1+j*m2+k*k1+l*k2+(1-i-j-k-l)*k5)
                    temp = (i,j,k,l,1-i-j-k-l)
                else:
                    val = mean_absolute_error(true,i*m1+j*m2+k*k1+l*k2+(1-i-j-k-l)*k5)
                    if val<best:
                        best = val
                        temp = (i,j,k,l,1-i-j-k-l)
                #print i,j,k,l,1-i-j-k-l,' : ',mean_absolute_error(true,i*m1+j*m2+k*k1+(1-i-j-k)*k2)
print best,temp

1119.06375239 (0.40000000000000002, 0.20000000000000001, 0.10000000000000001, 0.10000000000000001, 0.19999999999999993)


In [33]:
wt = np.array((0.40000000000000002, 0.20000000000000001, 0.10000000000000001, 0.10000000000000001, 0.19999999999999993))

In [34]:
cc = np.vstack((m1,m2,k1,k2,k5)).T

In [40]:
cur = np.sum(wt*cc,axis=1)
print mean_absolute_error(true,cur)

1119.06375239


In [41]:
tt = pd.read_csv('keras-test-1111.7.csv')
tt = tt.sort('id')
tt1 = pd.read_csv('keras-test-1114.csv')
tt1 = tt1.sort('id')
tt2 = pd.read_csv('keras-test-unk1.csv')
tt2 = tt2.sort('id')
tt3 = pd.read_csv('keras-test-unk2.csv')
tt3 = tt3.sort('id')
tt4 = pd.read_csv('keras-test-unk3.csv')
tt4 = tt4.sort('id')
m1 = pd.read_csv('xgb-test-1124.75.csv')['loss'].as_matrix()
m2 = pd.read_csv('xgb-test-1127.27.csv')['loss'].as_matrix()
k1 = tt['loss'].as_matrix()
k2 = tt1['loss'].as_matrix()
k3 = tt2['loss'].as_matrix()
k4 = tt3['loss'].as_matrix()
k5 = tt4['loss'].as_matrix()

x_test = np.vstack((m1,m2,k1,k2,k5)).T
test_pred = np.sum(wt*x_test,axis=1)
print test_pred.shape

/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/him/anaconda2/envs/gpu-tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


(125546,)


In [42]:
submission = pd.read_csv('../submission.csv')
submission.iloc[:, 1] = test_pred
submission.to_csv('Final_Submit_weighted_average.csv', index=None)

In [21]:
d_train = xgb.DMatrix(x_train,label=true)
watchlist = [(d_train, 'train')]

In [22]:
param = {'bst:max_depth':6, 'bst:eta':1, 'silent':1, 'objective':'reg:linear' }
param['nthread'] = 4
param['eval_metric'] = 'mae'

In [24]:
from sklearn.cross_validation import KFold

In [30]:
cv_sum = 0
n_fold = 20
kf = KFold(x_train.shape[0],n_folds=n_fold)
Test = xgb.DMatrix(x_test)
test_pred = np.zeros(x_test.shape[0])
for i,(inTr,inTe) in enumerate(kf):
    xtr, xval = x_train[inTr], x_train[inTe]
    ytr, yval = true[inTr], true[inTe]
    param = {'bst:max_depth':6, 'bst:eta':1, 'silent':0, 'objective':'reg:linear' }
    param['nthread'] = 4
    param['eval_metric'] = 'mae'
    d_train = xgb.DMatrix(xtr,label=ytr)
    d_test = xgb.DMatrix(xval,label=yval)
    watchlist = [(d_train, 'train'),(d_test, 'test')]
    clf = xgb.train(param,d_train,100,watchlist,early_stopping_rounds=1)
    print i+1,' Folds score: ',mean_absolute_error(clf.predict(d_test),yval)
    cv_sum += mean_absolute_error(clf.predict(d_test),yval)
    test_pred += clf.predict(Test)
    
print 'Total Average MAE: ',cv_sum/n_fold
test_pred /= n_fold

[0]	train-mae:2140.57	test-mae:2154.04
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 1 rounds.
[1]	train-mae:1592.53	test-mae:1604.03
[2]	train-mae:1312.35	test-mae:1323.47
[3]	train-mae:1185.85	test-mae:1201.91
[4]	train-mae:1134.33	test-mae:1155.33
[5]	train-mae:1116.85	test-mae:1141.67
[6]	train-mae:1113.5	test-mae:1141.67
Stopping. Best iteration:
[5]	train-mae:1116.85	test-mae:1141.67

1  Folds score:  1141.67046165
[0]	train-mae:2141.47	test-mae:2136.22
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 1 rounds.
[1]	train-mae:1593.36	test-mae:1588.14
[2]	train-mae:1313.61	test-mae:1310.34
[3]	train-mae:1187.15	test-mae:1190.8
[4]	train-mae:1135.67	test-mae:1142.94
[5]	train-mae:1118.12	test-mae:1128.48
[6]	train-mae:1114.94	test-mae:1127.92
[7]	train-mae:1116.81	test-mae:1131.79
Stopping. Best iteration:
[6]	train-mae:

In [32]:
sub = pd.read_csv('xgb-test-1124.75.csv')
sub['loss'] = test_pred
sub.to_csv('XGB-Stacking-6-Models.csv',index=False)

In [10]:
from sklearn.linear_model import Lasso
clfLasso = Lasso(fit_intercept=False)
clfLasso.fit(x_train,y)
print clfLasso.intercept_
print clfLasso.coef_
print mean_absolute_error(true,clfLasso.predict(x_train))

0.0
[ 0.53199188  0.19685441  0.37289664]
1137.04766454


/home/him/anaconda2/envs/theano-gpu/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [11]:
from sklearn.cross_validation import KFold
tot = 0
nfolds = 10
kf = KFold(x_train.shape[0],n_folds=nfolds)
for i, (train_index, test_index) in enumerate(kf):
    train_x,val_x = x_train[train_index], x_train[test_index]
    train_y,val_y = y[train_index], y[test_index]
    clfLasso = Lasso(fit_intercept=False,positive=False)
    clfLasso.fit(train_x,train_y)
    temp = mean_absolute_error(val_y,clfLasso.predict(val_x))
    print i," Fold error : ",temp
    print clfLasso.coef_
    tot += temp
    
tot /= nfolds * 1.0   
print 'Average MAE: ',tot

/home/him/anaconda2/envs/theano-gpu/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0  Fold error :  1147.83406141
[ 0.53608596  0.18970317  0.37629318]
1  Fold error :  1134.48614038
[ 0.531268    0.20029492  0.37156483]
2  Fold error :  1140.28040129
[ 0.5750781   0.14505795  0.38139682]
3  Fold error :  1123.93811606
[ 0.5524227   0.15984359  0.38996464]
4  Fold error :  1144.72708906
[ 0.52787838  0.19766014  0.37502009]
5  Fold error :  1138.58551793
[ 0.53374083  0.21554565  0.35199786]
6  Fold error :  1142.91081218
[ 0.50859194  0.20638012  0.38558806]
7  Fold error :  1139.91617559
[ 0.5342602   0.1989915   0.36916207]
8  Fold error :  1127.09394357
[ 0.52243648  0.22353138  0.35560687]
9  Fold error :  1131.20893884
[ 0.49823     0.23103579  0.37279803]
Average MAE:  1137.09811963


In [12]:
tt = pd.read_csv('keras-test-1111.7.csv')
tt = tt.sort('id')
tt1 = pd.read_csv('keras-test-single.csv')
tt1 = tt1.sort('id')
m1 = pd.read_csv('xgb-test-1124.75.csv')['loss'].as_matrix()
m2 = pd.read_csv('xgb-test-1127.27.csv')['loss'].as_matrix()
m3 = pd.read_csv('xgb-test-single.csv')['loss'].as_matrix()
k1 = tt['loss'].as_matrix()
k2 = tt1['loss'].as_matrix()
x_test = np.vstack((m1,m2,k1)).T

/home/him/anaconda2/envs/theano-gpu/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
/home/him/anaconda2/envs/theano-gpu/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [39]:
pred = clfLasso.predict(x_test)

In [40]:
pred = clfLasso.predict(x_test)
sub = pd.read_csv('xgb-test-1124.75.csv')
sub['loss'] = pred
sub.to_csv('Lasso-Stacking-4 way.csv',index=False)

In [13]:
import numpy as np
np.random.seed(123)

import pandas as pd
import subprocess
from datetime import datetime
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.callbacks import CSVLogger,EarlyStopping, ModelCheckpoint

def nn_model():
    model = Sequential()
    
    model.add(Dense(4, input_dim = 4))
    model.add(PReLU())
    model.add(BatchNormalization())
        
    model.add(Dense(4, init = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())    
    
    model.add(Dense(4, init = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())    
    
    model.add(Dense(1, init = 'he_normal'))
    model.compile(loss = 'mae', optimizer = 'adam')
    return(model)

In [15]:
model = nn_model()
model.fit(x_train,y,nb_epoch=10,batch_size=128)

Epoch 1/10
188318/188318 [==============================] - 10s - loss: 3028.1670    
Epoch 2/10
188318/188318 [==============================] - 10s - loss: 2986.8705    
Epoch 3/10
188318/188318 [==============================] - 10s - loss: 2918.5364    
Epoch 4/10
188318/188318 [==============================] - 10s - loss: 2828.3863    
Epoch 5/10
188318/188318 [==============================] - 11s - loss: 2718.5228    
Epoch 6/10
188318/188318 [==============================] - 10s - loss: 2590.1491    
Epoch 7/10
188318/188318 [==============================] - 9s - loss: 2445.1997     
Epoch 8/10
188318/188318 [==============================] - 7s - loss: 2286.5916     
Epoch 9/10
188318/188318 [==============================] - 7s - loss: 2117.8033     
Epoch 10/10
188318/188318 [==============================] - 8s - loss: 1942.8898     
